#Model to Classify whether on-invoice discount is zero or not

Importing libraries

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import plot_importance
from matplotlib import pyplot

Upload "Data_Encoded.csv" from the INPUT folder provided in the drive link

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Data_Encoded.csv to Data_Encoded.csv


In [4]:
data_c = pd.read_csv('/content/Data_Encoded.csv')
data_c.head()

,total_volume_2019,total_volume_2018,city_tier,gross_turnover_2019,on_invoice_discount,off_invoice_discount,product_volume_2019,tax,total_discount,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur,province_Walloon Brabant
0,0.48,0.557,1,141.20,0.00,0.0,0.48,12.23,0.00,128.97,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.45,0.540,2,113.65,0.00,0.0,0.45,7.85,0.00,105.80,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,270.97,225.720,2,61551.06,14511.46,0.0,238.57,3577.69,14511.46,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,270.97,225.720,2,6001.00,1425.19,180.0,20.40,519.41,1605.19,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,270.97,225.720,2,3051.36,718.32,0.0,12.00,288.63,718.32,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Creating a column which tells if the poc got an on-invoice discount or not

In [5]:
data_c['ondiscount']=0

In [6]:
for i in range(len(data_c)):
  if data_c['on_invoice_discount'][i]>0:
    data_c['ondiscount'][i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
data_c.head(10)

,total_volume_2019,total_volume_2018,city_tier,gross_turnover_2019,on_invoice_discount,off_invoice_discount,product_volume_2019,tax,total_discount,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur,province_Walloon Brabant,ondiscount
0,0.48,0.557,1,141.20,0.00,0.00,0.48,12.23,0.00,128.97,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0.45,0.540,2,113.65,0.00,0.00,0.45,7.85,0.00,105.80,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,270.97,225.720,2,61551.06,14511.46,0.00,238.57,3577.69,14511.46,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,270.97,225.720,2,6001.00,1425.19,180.00,20.40,519.41,1605.19,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,270.97,225.720,2,3051.36,718.32,0.00,12.00,288.63,718.32,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
5,29.61,25.860,4,104.10,0.00,0.00,0.30,1.55,0.00,102.55,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,29.61,25.860,4,176.50,0.00,0.00,0.60,15.27,0.00,161.23,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,29.61,25.860,4,2227.65,0.00,686.13,11.34,61.37,686.13,2166.28,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,29.61,25.860,4,3412.19,0.00,1050.98,17.37,94.12,1050.98,3318.07,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,0.42,0.505,4,20.82,6.15,0.00,0.06,0.31,6.15,20.51,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


Dropping redundant columns/ Getting the data ready to train

In [8]:
data_c['2019-2018'] = data_c['total_volume_2019']-data_c['total_volume_2018']

In [9]:
data_c.drop('total_volume_2018', axis = 1, inplace = True)

In [10]:
data_c.drop(['total_discount', 'on_invoice_discount', 'off_invoice_discount', 'gross_turnover_2019','tax'], axis = 1, inplace = True)

In [11]:
X_ondc = data_c.drop('ondiscount', axis = 1)
Y_ondc = data_c['ondiscount']

Splitting the data for training and testing

In [12]:
X_ondc_train, X_ondc_test, y_ondc_train, y_ondc_test = train_test_split(X_ondc,Y_ondc,test_size = 0.2, random_state = 10, stratify = Y_ondc)

Final training model after hyperparameter tuning

In [13]:
XGBc = XGBClassifier(base_score=0.5, booster='gbtree', boosting='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.25, learning_rate=0.2, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=2, seed=None,
              silent=None, subsample=1, verbosity=1, scoring = 'roc_auc')
XGBc.fit(X_ondc_train,y_ondc_train, eval_set= [(X_ondc_test,y_ondc_test)], early_stopping_rounds=10, eval_metric= 'aucpr')

[0]	validation_0-aucpr:0.873119
Will train until validation_0-aucpr hasn't improved in 10 rounds.
[1]	validation_0-aucpr:0.886886
[2]	validation_0-aucpr:0.902792
[3]	validation_0-aucpr:0.906526
[4]	validation_0-aucpr:0.914797
[5]	validation_0-aucpr:0.922024
[6]	validation_0-aucpr:0.928679
[7]	validation_0-aucpr:0.933818
[8]	validation_0-aucpr:0.939122
[9]	validation_0-aucpr:0.940789
[10]	validation_0-aucpr:0.949002
[11]	validation_0-aucpr:0.95083
[12]	validation_0-aucpr:0.953167
[13]	validation_0-aucpr:0.954797
[14]	validation_0-aucpr:0.956275
[15]	validation_0-aucpr:0.957406
[16]	validation_0-aucpr:0.957934
[17]	validation_0-aucpr:0.961669
[18]	validation_0-aucpr:0.961955
[19]	validation_0-aucpr:0.963024
[20]	validation_0-aucpr:0.963546
[21]	validation_0-aucpr:0.966543
[22]	validation_0-aucpr:0.966737
[23]	validation_0-aucpr:0.966793
[24]	validation_0-aucpr:0.967048
[25]	validation_0-aucpr:0.968524
[26]	validation_0-aucpr:0.968276
[27]	validation_0-aucpr:0.968183
[28]	validation_0-auc

XGBClassifier(base_score=0.5, booster='gbtree', boosting='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.25, learning_rate=0.2, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=2, scoring='roc_auc',
              seed=None, silent=None, subsample=1, verbosity=1)

Predicting test dataset

In [ ]:
XGBcpred = XGBc.predict(X_ondc_test)

Accuracy score

In [ ]:
metrics.accuracy_score(y_ondc_test, XGBcpred)

0.9127914466056937

Confusion matrix

In [ ]:
metrics.confusion_matrix(y_ondc_test, XGBcpred)

array([[1564,  563],
       [ 114, 5522]])

Precision and Recall scores

In [ ]:
print(metrics.precision_score(y_ondc_test, XGBcpred))
print(metrics.recall_score(y_ondc_test, XGBcpred))

0.9074774034511093
0.9797728885734563


Downloading the trained model as pickle file ( commented )

In [ ]:
# import pickle
# with open('XGB classify on invoice discount.pickle','wb') as modelFile:
#      pickle.dump(XGBc,modelFile)

#Hyper parameter tuning (Commented)

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# n_estimators = [int(x) for x in np.linspace(start = 250, stop=500, num = 10)]
# learning_rate = [0.1,0.2,0.5]
# boosting = ['gbtree']
# max_depth = [int(x) for x in np.linspace(10,15,3)]
# min_child_weight = [1]
# scale_pos_weight = [2,2.5]
# gamma = [0.2,0.25,0.3]
# random_grid = {'n_estimators': n_estimators,
#                'learning_rate': learning_rate,
#                'boosting': boosting,
#                'max_depth': max_depth,
#                'min_child_weight': min_child_weight,
#                'scale_pos_weight': scale_pos_weight,
#                'gamma': gamma,
#                }
# print(random_grid)

In [ ]:
# h_xgb = XGBClassifier()
# h_xgb_rscv = RandomizedSearchCV(estimator=h_xgb, param_distributions= random_grid,n_iter=10, cv=3,verbose =1, random_state=69)
# h_xgb_rscv.fit(X_ondc_train,y_ondc_train)

In [ ]:
# h_xgb_rscv.best_estimator_